# Colour addition workflow

You are given the imports below for all the necessary modules and classes required to run our example workflow

In [3]:

import time

from ochra_discovery.storage.vessel import Vessel
from ochra_discovery.storage.reagent import Reagent
from ochra_discovery.equipment.operation import Operation
from ochra_discovery.equipment.operation_result import OperationResult
from ochra_discovery.spaces.lab import Lab
from ochra_discovery.storage.holder import Holder

from ika_rct_digital.device import IkaPlate
from tecan_xcalibur.device import TecanXCalibur
from webcam.device import Webcam
from abb_yumi.device import AbbYuMi
from xpr_quantos.device import XprQuantos
from kuka_kmriiwa.device import KukaKMRiiwa

## Connecting to the lab server
In order to run your experiment in the lab, first you need to connect to the lab server. In the code box below, using `Lab` class, connect to the server at address *127.0.0.1:8001* with an experiment id of your choice.

In [6]:
# connect to lab server
lab = Lab("127.0.0.1:8001","asd123")

## Retrieving a station
To access the devices available in our lab, we need to get the station which the device belongs to. Below use `get_station` method from the `Lab` class to retrieve *yumi_station*

In [8]:
# get station
yumi_station = lab.get_station("yumi_station")

HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /lab/stations/get?identifier=yumi_station (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000180F8B2AAE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


LabEngineException: Request Failed: HTTPConnectionPool(host='127.0.0.1', port=8001): Max retries exceeded with url: /lab/stations/get?identifier=yumi_station (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000180F8B2AAE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Defining labware
To run our experiments, we need to define the used labware, such as the used vials and racks. In the code box below, define a rack that can hold up to 4 vials using the class `Holder`. Moreover, using the `Vessel` class, create 4 empty vials that have a maximum capacity of 15 ml

In [ ]:
# construct the rack holder
rack = Holder(type="rack", max_capacity=4)

# construct and add vials to the rack
for i in range(8):
    vial = Vessel(type="vial", max_capacity=15, capacity_unit="ml")
    rack.add_container(vial)

## Adding labware to the station
The created labware needs to be added to our station's inventory so it can be used in the experiment. Below use `add_container` method of the `inventory` attribute of our station to do so

In [ ]:
# add the rack to the station inventory
yumi_station.inventory.add_container(rack)

## Workflow steps
---

### Workflow step (1) - Pick a vial and decap it
The first step in our workflow involves using the YuMi robot to decap one of the vials before adding liquid to it. Use `get_robot` method of our station to retrieve our robot named *yumi*

*hint: use python type hints when getting a device to have access to its methods using autocomplete*

In [ ]:
# get the robot from the station
yumi: AbbYuMi = yumi_station.get_robot("yumi")

Using the retrieved robot, call its `execute` method to execute the task name *pick_vial_from_rack* with the argument: {"index":1}, so the robot pick a vial from the input rack and place it in the operation zone

In [ ]:
# execute robot task
yumi.execute("pick_vial_from_rack", {"index":1})

Similarly, execute the task *uncap_vial* with no arguments

In [ ]:
# decap the vial
yumi.execute("uncap_vial", {})

### Workflow step (2) - Adding liquids to the vial
We need to retrieve the Tecan XCalibur liquid handler from our station so we can use it functionalities. Similar to how we needed to retrieve the YuMi robot from the station, use `get_device` method to retrieve our liquid handler named *xcalibur_pump*

*hint: don't forget using type hinting*

In [ ]:
# get the liquid handler from the station
xcalibur: TecanXCalibur = yumi_station.get_device("xcalibur_pump")

Next, load the vial into the liquid handler by executing the robot task *place_vial_in_pump* with no arguments

In [ ]:
# load the vial into the liquid handler
yumi.execute("load_tecan_pump", {})

Use the liquid handler `dispense` method to add 10ml of water to the loaded vial

In [ ]:
# dispense 10ml of water into the vial
xcalibur.dispense(reagent="water", volume=10, volume_unit="ml")

Currently in our framework, all the operations are defined explicitly. In other words, adding liquid to the vial needs to be explicitly defined by the programmer. In the code box below, define the added reagent using the `Reagent` class and use `add_reagent` method of our vial to define the addition

In [ ]:
# define the added reagent
water = Reagent("water", 10, "ml")

# add the added reagent to the vial
vial.add_reagent(water)

Similarly, use the liquid handler `dispense` method to add 3ml of dye to the loaded vial and explicitly define the reagent addition

In [ ]:
# dispense dye into the vial
xcalibur.dispense(reagent="dye", volume=3, volume_unit="ml")

# define the added reagent
dye = Reagent("dye", 3, "ml")

# add the added reagent to the vial
vial.add_reagent(dye)

Finally, use the YuMi robot to unload the vial from the liquid handler and place it back in the operation zone using the task *unload_tecan_pump* before capping the vial with the task *cap_vial*

In [ ]:
# unload the vial from the liquid handler
yumi.execute("unload_tecan_pump", {})

# cap the vial
yumi.execute("cap_vial", {})

### Workflow step (3) - Mixing the liquids
In order to mix the added liquids, we need to use our IKA RCT digital plate. Similar to the previous steps, first retrieve the device from the station using its given name *ika_plate*

In [ ]:
# get the ika plate from the station
ika_plate: IkaPlate = yumi_station.get_device("ika_plate")

Use the YuMi robot to load the vial into the IKA plate using the task *load_ika_plate*, stir the liquids for 10 seconds using the methods `start_stirring` and `stop_stirring` in the IKA plate. To add the delay, you can use `sleep` method from python `time` library. Finally, make the robot unload the IKA plate using the task *unload_ika_plate*

In [ ]:
# load the vial into the ika plate
yumi.execute("load_ika_plate", {})

# start stirring
ika_plate.start_stirring()

# sleep for 10 seconds
time.sleep(10)

# stop stirring
ika_plate.stop_stirring()

# unload the vial from the ika plate
yumi.execute("unload_ika_plate", {})

### Workflow step (4) - Image the resulting solution
The final step in our workflow is to take a picture of the vial to image the resulting solution before running further analysis. Like in the previous steps, first retrieve the camera device from the station using its given name *camera*. Afterwards, use the YuMi robot to present the vial to the camera using the task *goto_camera_pose*

In [ ]:
# get the camera from the station
camera: webcam = yumi_station.get_device("camera")

# present the vial to the camera
yumi.execute("goto_camera_pose", {})

Next, use the camera `take_image` method to take a picture of the presented vial. Note that, all device and robot methods return `Operation` object upon calling. Store this object by assigning a variable named `op` to the method call.

*hint: use type hinting on the returned object to have access to its attributes and methods

In [ ]:
# take an image of the vial
op: Operation = camera.take_image()

Finally, use the YuMi robot to return the vial to the operation zone and then to place it back in the rack using the tasks *return_from_camera_pose* and *place_vial_in_rack* respectively. The latter task takes arguments {"index":1}

In [ ]:
# return the vial from the camera
yumi.execute("return_from_camera_pose", {})

# place the vial back in the rack
yumi.execute("place_vial_in_rack", {"index":1})

---
## Retrieve experiment data
After running our workflow, we want to access the image that was taken during the experiment to further analyze our data. To access the results of the operation, use the method `get_result_object` to retrieve `OperationResult` that provide access to the results data

In [ ]:
# get the operation result object
img_result: OperationResult = op.get_result_object()

Use the method `save_data` of the result object to store data to the disk under the path /tmp/images, so we can access it later

In [ ]:
# store data to disk
img_result.save_data("/tmp/images/img_1.png")

## Using 3rd party libraries
The result data can be directly accessed in runtime to run further analysis on it using 3rd party libraries or to convert it to another format if needed. Retrieve the result data directly using the method `get_data` from the OperationResult object.Afterwards, import `cv2` library and use it to display the taken image and then convert it to grey-scale colour space using the appropriate cv2 methods

In [ ]:
# import the 3rd party library
import cv2

# get the result data
img = img_result.get_data()

# display the image and convert to grey-scale
cv2.imshow("Image", img)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray Image", gray_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
